# Curso de Estadística Bayesiana


#  Regresión Robusta

En este cuaderno veremos como utilizar un comando mágico para poder tener celdas completas con código STAN.

## Autor

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

## Referencias

1. Basado en <url>https://jrnold.github.io/bayesian_notes/</url>

#  Distribuciones de cola pesada


La regresión lineal Bayesiana con errores distribuidos normalmente es sensible a los valores atípicos. Esto se debe a que la distribución normal tiene probabilidades de cola estrechas, con aproximadamente el 99.8% de la probabilidad dentro de tres desviaciones estándar.

La regresión robusta se refiere a métodos de regresión que son menos sensibles a los valores atípicos. La regresión robusta Bayesiana usa distribuciones con colas más pesadas que las normales.

La sigiuente gráfica mustra las funciones de densidad de probabilidad (pdf)  de las distribuciones normal, doble exponencial (Laplace) y Student-t $ (df = 4) $ todas con media 0 y escala 1,




In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import laplace
#
fig, ax = plt.subplots(1, 1,figsize=(12, 8))
plt.title('Funciones de densidad',fontsize=14)

x = np.linspace(-6,6,300)
#
y_laplace = laplace.pdf(x)
ax.plot(x, y_laplace, 'g-', lw=2, alpha=0.6, label='laplace pdf')
#
y_norm = norm.pdf(x)
ax.plot(x, y_norm, 'r-', lw=2, alpha=0.6, label='norm pdf')
#
y_student_t = t.pdf(x, df=10)
ax.plot(x, y_student_t, 'b-', lw=2, alpha=0.6, label='student-t pdf')

#
plt.xlabel('$x$',fontsize=14)
plt.ylabel('$pdf(x) ',fontsize=14)
plt.legend()
plt.show()

La siguiente gráfica muestra las funciones de información (diferencial), definida por $-\log pdf(x)$. Esta muestra el **grado de sorpresa** de observar el valor en cada punto.


Tanto la distribución de Student como la doble exponencial tienen valores sorpresa muy por debajo de lo normal en los rangos (-6, 6). Esto significa que los valores atípicos tendrán menos efecto en el log-posterior de los modelos que usan estas distribuciones. La línea de regresión necesitaría moverse menos para incorporar esas observaciones, ya que la distribución del error no las considerará inusuales.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import laplace
#
fig, ax = plt.subplots(1, 1,figsize=(12, 8))
plt.title('Funciones de información',fontsize=14)

x = np.linspace(-6,6,300)
#
y_laplace = -np.log(laplace.pdf(x))
ax.plot(x, y_laplace, 'g-', lw=2, alpha=0.6, label='laplace pdf')
#
y_norm = -np.log(norm.pdf(x))
ax.plot(x, y_norm, 'r-', lw=2, alpha=0.6, label='norm pdf')
#
y_student_t = -np.log(t.pdf(x, df=10))
ax.plot(x, y_student_t, 'b-', lw=2, alpha=0.6, label='student-t pdf')

#
plt.xlabel('$x$',fontsize=14)
plt.ylabel('$I(x) = -\log \ f(x)$',fontsize=14)
plt.legend()
plt.show()

# Distribución t-Student


El modelo Bayesiano más utilizado para la regresión robusta es una regresión lineal con errores  independientes Student-$t$ (Geweke 1993; A. Gelman, Carlin, et al. 2013, Ch. 17)
    
    
$$
y_i \sim t\text{-Student}\left(\nu, \mu_i, \sigma \right)
$$
    
$ \nu \in  \mathcal{R}^{+} $ es un parámetro de grados de libertad, $ \mu_i \in \mathcal{R} $ son ubicaciones específicas de observación a menudo modeladas con una regresión, y $ \sigma \in \mathcal{R} ^ {+} $ es un parámetro de escala. La fdp de la distribución $ t $-Student  está dado por
    
$$
t\text{-Student} \left(x; \nu, \mu, \sigma \right) = K \left(1 + \frac {1}{\nu} \left(\frac {x- \mu}{\sigma} \right)^2 \right)^{- \frac{\nu + 1} {2}},
$$

donde $ K $ es una constante de normalización.

    
Tenga en cuenta que cuando $ \nu \to \infty $, este modelo se aproxima a un modelo normal independiente, ya que la distribución de $ t $-Student se aproxima asintóticamente a la distribución normal a medida que aumentan los grados de libertad. Para el valor de $ \nu $, se puede usar un bajo grado de libertad, digamos $ \nu \in (4, 6) $, o se le puede asignar una distribución previa.



Para la distribución de $t$-Student, la existencia de varios momentos depende del valor de $ \nu $: la media existe para $ \nu \ge 1 $ varianza para $ \nu \ge 2 $, y la curtosis para $ \nu \ge 3 $.

Como tal, a menudo es útil restringir el soporte de $ \nu $ a al menos 1 o 2 (o incluso más) para garantizar la existencia de una media o variación.

Una distribución previa razonable para el parámetro de grados de libertad es un Gamma
distribución con el parámetro de forma 2 y un parámetro de escala inversa (velocidad) de 0.1 (Juárez and Steel 2010, @Stan-prior-choices), $\nu \sim Gamma(2, 10)$.

In [ ]:
# gráficos de la distribución Gamma
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gamma


x = np.linspace(0,50,50)             
fig, ax = plt.subplots(1, 1, figsize=(12,8))
alpha = 2
beta  = 10
y = gamma.pdf(x, alpha, loc=0, scale=beta)
ax.set_ylim(0,0.04)
ax.plot(x,y)
ax.plot(0, 0, label="$\\alpha$ = {:3.2f}\n$\\beta$ = {:3.2f}".format(alpha, beta), alpha=0)
ax.legend(fontsize=12)
ax.set_xlabel('$\\nu$', fontsize=14)
ax.set_ylabel('$pdf(x)$', fontsize=14)
fig.suptitle('Función de densidad de la distribución Gamma(2,10)',fontsize=16)
plt.show()
    

Esta densidad coloca la mayoría de la masa anterior para valores $\nu \le 50$, en los que la distribución de $t$-Student es sustancialmente diferente de la distribución Normal, y también permite que existan todos los momentos anteriores.

# Datos

Algunos autores han debatido si la política y la economía de la izquierda han tenido un impacto sostenido en el crecimiento económico de las democracias capitalistas relativamente ricas desde 1973. Los datos se refieren al crecimiento de algunos países entre 1978 y 1982. El autor afirma que el crecimiento acelerado se debe ambos: los sindicatos eran organizacionalmente fuertes y los partidos de izquierda eran fuertes participantes en los gobiernos.


In [ ]:
import pandas as pd
import numpy as np
path = "./datos/"
econ_growth = pd.read_csv(path +"econ_growth.csv")

In [ ]:
econ_growth.head()

In [ ]:
econ_growth.describe().T

In [ ]:
# add a new column to the data frame. cross effect labor_org*social_dem
econ_growth['labor_soc_dem'] = econ_growth.labor_org * econ_growth.social_dem
econ_growth.head()

In [ ]:
# normalizing the columns of data
# install sklearn
# pip install sklearn
from sklearn.preprocessing import StandardScaler as SS

econ_growth['growth_ss'] = SS().fit_transform(econ_growth[['econ_growth']])
econ_growth['labor_ss'] = SS().fit_transform(econ_growth[['labor_org']])
econ_growth['soc_dem_ss'] = SS().fit_transform(econ_growth[['social_dem']])
econ_growth['labor_soc_dem_ss'] = SS().fit_transform(econ_growth[['labor_soc_dem']])
econ_growth.head()

## Instalando el comando mágico de pystan

Este comando permite escribir código Stan directamente en una celda. Lo primero que hay que hace es  instalar el comando. En la siguiente código se muestra una forma de hacer la instalación.

Para detalles vea el sitio [stan-magic]( https://github.com/Arvinds-ds/stanmagic/blob/master/StanMagic-Help.ipynb)

In [ ]:
import pystan
%load_ext stanmagic

In [ ]:
%%stan -f Robust_reg.stan
// Linear Model with Student-t Errors
data {
  // number of observations
  int N;
  // response
  vector[N] y;
  // number of columns in the design matrix X
  int K;
  // design matrix X
  // should not include an intercept
  matrix [N, K] X;
  // priors on alpha
  real scale_alpha;
  vector[K] scale_beta;
  real loc_sigma;
  // keep responses
  int use_y_rep;
  int use_log_lik;
}
parameters {
  // regression coefficient vector
  real alpha;
  vector[K] beta;
  real sigma;
  // degrees of freedom;
  // limit df = 2 so that there is a finite variance
  real nu;
}
transformed parameters {
  vector[N] mu;

  mu = alpha + X * beta;
}
model {
  // priors
  alpha ~ normal(0.0, scale_alpha);
  beta ~ normal(0.0, scale_beta);
  sigma ~ exponential(loc_sigma);
  // see Stan prior distribution suggestions
  nu ~ gamma(2, 0.1);
  // likelihood
  y ~ student_t(nu, mu, sigma);
}
generated quantities {
  // simulate data from the posterior
  vector[N * use_y_rep] y_rep;
  // log-likelihood posterior
  vector[N * use_log_lik] log_lik;
  for (i in 1:num_elements(y_rep)) {
    y_rep[i] = student_t_rng(nu, mu[i], sigma);
  }
  for (i in 1:num_elements(log_lik)) {
    log_lik[i] = student_t_lpdf(y[i] | nu, mu[i], sigma);
  }
}

In [ ]:
_stan_model

In [ ]:
from collections import OrderedDict
#data = OrderedDict({'X':X, 'y': y, 'N':N, 'K': K})

import numpy as np
#X= np.array(econ_growth[['labor_ss', 'soc_dem_ss','labor_soc_dem_ss']])
#y= np.array(econ_growth[['growth_ss']])
data = OrderedDict({'N':15, 'K':3, 'scale_alpha': 10, 'scale_beta': [2.5, 2.5, 2.5], 
                    'loc_sigma':  1, 'use_y_rep': 1, 'use_log_lik': 1, 'd': 4,
                    'X':np.array(econ_growth[['labor_ss', 'soc_dem_ss','labor_soc_dem_ss']]), 
                    'y': np.reshape(np.array(econ_growth[['growth_ss']]),15,0)})


In [ ]:
econ_growth

In [ ]:
data

In [ ]:
robust_reg_model = pystan.StanModel(file=_stan_model.model_file)

In [ ]:
robust_reg_sample = robust_reg_model.sampling(data=data)

In [ ]:
robust_reg_sample